In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from brainpipe.classification import *

from brainpipe.statistics import *
import scipy.io as sio
import mne
from mne.viz import plot_topomap
from mne import pick_types, find_layout
import random as rand


# Import files

In [4]:
grp = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9', 'E10', 'E11', 'E12', 'E13', 'E14', 'E15', 'E16',
           'E17', 'E18', 'E19', 'E20', 'E21', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E28', 'E29', 'E30', 'E31',
           'E32', 'E33', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E41', 'E42', 'E43', 'E44', 'E46',
           'E47', 'E48', 'E49', 'E50', 'E51', 'E52', 'E53', 'E54', 'E55', 'E56', 'E57', 'E58', 'E59', 'E60', 'E61',
           'E62', 'E63','E64','E65']

In [ ]:
filepath = "/media/karim/4TB_drive/Final_DFA_analysis/60sec/"
savepath = "/media/karim/4TB_drive/Final_DFA_analysis/60sec/classif/classif_intra_final/"
sensors_pos = sio.loadmat('/home/thomasthiery/Dropbox/Rep_MATLAB/electrodes_sevo_new.mat')['electrodes_sevo_new']

Cx = 'beta'
subjects = ['Sub_S2', 'Sub_S3','Sub_S4','Sub_S5','Sub_S6','Sub_S8','Sub_S10']
temp_con = '_conscious_'
temp_uncon = '_unconscious_'


for k in subjects:

    unconscious = sio.loadmat(filepath + k + temp_con   + Cx +  '_dfa_60sec_5_50sec.mat') ['beta']
    conscious_temp = sio.loadmat(filepath + k + temp_uncon  + Cx +  '_dfa_60sec_5_50sec.mat') ['beta']
    assert conscious_temp.shape[0]==unconscious.shape[0],"attention, con et uncon pas le même nb d'elec"
    if conscious_temp.shape[0] == 65:
        elec = np.ones(shape = (conscious_temp.shape[0]),dtype = bool)
        elec[[39,44]] = False
        conscious_temp = conscious_temp[elec,:]
        unconscious = unconscious[elec,:]
    
    conscious=np.zeros((len(unconscious),len(unconscious[1]))) 
    for jj in range(len(conscious)):
        conscious[jj,:] = np.random.choice(conscious_temp[jj],len(unconscious[1]))

    a = np.zeros((conscious.shape[1]))
    b = np.ones((unconscious.shape[1]))
    
    conscious_unconscious = np.concatenate((conscious, unconscious),axis=1).T

    vectorlabel = np.concatenate((a, b), axis=0)
    
    cvO = defCv(vectorlabel, rep=1)
    #print(cvO)

    clfO = defClf(vectorlabel, clf='lda')
    #print(clfO)

    cl = classify(vectorlabel, clf=clfO, cvtype=cvO)
    #print(cl)
    
    da, pvalue, daperm = cl.fit(conscious_unconscious, n_perm=1, grp=grp , method='label_rnd')

    sio.savemat(savepath+k+'_da_'+Cx+'_new_sf_shallow_final_intra',{'da': da})
    sio.savemat(savepath+k+'_pvalue_'+Cx+'_new_sf_shallow_final_intra',{'pvalue': pvalue})
    sio.savemat(savepath+k+'_da_perm_'+Cx+'_new_sf_shallow_final_intra',{'daperm': daperm})
    print(k)
    fig1 = plt.figure(1, figsize=(20,10))

    cl.daplot(da, daperm=daperm, chance_method='perm', rmax=['top', 'right'],
            dpax=['bottom', 'left'], cmap='magma', ylim=[40,100], chance_unique=True,chance_level = 0.01,
            chance_color='darkgreen')
    fig1.savefig(savepath+k+'_'+Cx+'_new_sf_shallow_final.png', dpi=300, bbox_inches='tight')
    
    # Display informations about features :
    cl.info.to_excel(savepath+k+'_'+Cx+'_new_sf_shallow_final_intra.xlsx')
    plt.clf()
    plt.close()
    data = sio.loadmat(savepath+k+'_da_'+Cx+'_new_sf_shallow_final_intra.mat')['da'].T 
    data = np.squeeze(data)

    mask_default = np.full((len(data)), False, dtype=bool)
    mask = np.array(mask_default)
    mask[data >np.amax(daperm)-0.01] = True

    mask_params = dict(marker='*', markerfacecolor='w', markersize=15) # significant sensors appearence

    fig = plt.figure(figsize = (10,15))
    plt.subplot(1,2,1)
    ax,_ = plot_topomap(data, sensors_pos,
                    cmap='inferno', show=False, vmin=40, vmax=70, mask = mask, mask_params = mask_params)

    fig.colorbar(ax, shrink=0.25)
    plt.savefig(savepath + k+ '_da_sf_shallow_final_'+Cx+'_fig_python.png', dpi = 300)
    plt.clf()
    plt.close()
    del da, pvalue, cl, fig, fig1